In [67]:
# Python libraries:
import os
import sys
import re
from dateutil import parser
# import datetime
from datetime import datetime
from datetime import date
import builtins
import json
import functools
import operator
from itertools import product

# Numpy & Pandas!
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format
pd.options.display.max_columns = None

# Spark!
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
from pyspark.sql import SparkSession, Row

if 1:
    spark = SparkSession.builder\
    .master("local[*]")\
    .appName("myapp").getOrCreate()
# else:
#     spark = SparkSession.builder.master("yarn")\
#     .config("spark.executor.instances", "32")\
#     .config("spark.executor.cores", "4")\
#     .config("spark.executor.memory", "4G")\
#     .config("spark.driver.memory", "4G")\
#     .config("spark.executor.memoryOverhead","4G")\
#     .config("spark.yarn.queue","Medium")\
#     .appName("myapp")\
#     .getOrCreate()

sc = spark.sparkContext
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")
spark.conf.set("spark.debug.maxToStringFields","true")

In [68]:
%load_ext autoreload
%autoreload 2
# The autoreload extension is already loaded. To reload it, use:
#  %reload_ext autoreload


# mylib:
my_library = os.path.expanduser('~/.myconfigs')
my_spark = os.path.expanduser('~/spark2_dfanalysis')
sys.path.append(my_library)
sys.path.append(my_spark)


from shared.app_context import *
from builder.DataFrameBuild import *

ctx = ApplicationContext("Dev-Job")

DFB = DataFrameBuild(ctx.spark)

print("%16s  %s" % ("Python Version:",sys.version))
print("%16s  %s" % ("Python Path:",os.path.dirname(sys.executable)))
print("%16s  %s" % ("My Python Libs:",my_library))
print("%16s  %s" % ("My Spark Dir:",my_spark))
print("%16s  %s" % ("My Spark Ctx:",ctx.spark))
# print(ctx.spark)
# print(os.listdir(my_spark))
# print(sys.path)
# print("\n")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
 Python Version:  3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]
    Python Path:  C:\Users\d810216\AppData\Local\conda\conda\envs\my_root
 My Python Libs:  C:\Users\d810216/.myconfigs
   My Spark Dir:  C:\Users\d810216/spark2_dfanalysis
   My Spark Ctx:  <pyspark.sql.session.SparkSession object at 0x000001DD4930CBA8>


# Read / Write / Partition:

In [69]:
# Let's build 10 dataframes!
num = 500

df10 = []

for x in range(10):
    df = DFB.arrays_to_dataframe(
        [DFB.build_array("string",num=num,width=8),
         DFB.build_array("integer",num=num,nrange=(1,4)),
         DFB.build_array("integer",num=num,nrange=(1,12)),
         DFB.build_array("double",num=num,nrange=(0.0,10000))],
        ['passwords','quarter','month','price'])
    df10.append(df)
    
    
# for df in df10:
#     df.show(4)

In [74]:
output_dir = os.path.join(my_spark,'data')
print(output_dir)

for i,df in enumerate(df10):
    
    print(df.count())
    num = str(i).zfill(2)
    
    num_dir = os.path.join(output_dir,num)
    #num_dir = str("file:///") + str(os.path.join(output_dir,num))
    num_dir = os.path.join("file:///http://localhost:8888/tree/spark2_dfanalysis/","data",num)
    print(num_dir)
    
#     df.coalesce(2).write.parquet(num_dir,mode="overwrite")
    df.write.parquet(num_dir,mode="overwrite")

    
    

C:\Users\d810216/spark2_dfanalysis\data
500
file:///http://localhost:8888/tree/spark2_dfanalysis/data\00


Py4JJavaError: An error occurred while calling o1084.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:224)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:154)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOper

In [ ]:
dfi = file_io.read.parquet(config['source'][cont_panel])
dfo = file_io.read.parquet(config['target'][cont_panel])

In [ ]:
lst_files = get_file_list(config['source'][cont_panel])
lst_dates = sorted([x.split("=")[-1] for x in lst_files],reverse=True)

In [ ]:
# Writing!

# for date in lst_dates:
#     print(date)
#     fn = "%s/fiscal_week=%s" % (config['source'][cont_panel],date)
#     fn1 = "%s/fiscal_week=%s" % (config['target'][cont_panel],date)
#     #print(fn)
#     df = spark.read.parquet(fn)
#     df1 = df.withColumnRenamed("household_id","ehhn")
#     print(df.count())    
#     df1.coalesce(1).write.parquet(fn1,mode="overwrite")
#     break

In [ ]:
for date in lst_dates:
    #print(date)
    fn = "%s/fiscal_week=%s" % (config['source'][cont_panel],date)
    fn1 = "%s/fiscal_week=%s" % (config['target'][cont_panel],date)
    #print(fn)
    df = spark.read.parquet(fn)
    df1= spark.read.parquet(fn1)
    c0 = df.count()
    c1 = df1.count()
    print("Date: %s %11d %11d %7d" % (date,c0,c1,c0-c1))